In [37]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
import time

In [38]:
def load_data(filepath, sep=';', encoding='utf-8'):
    """Charge les données avec gestion des erreurs améliorée"""
    try:
        df = pd.read_csv(filepath, sep=sep, engine='python', quoting=3, 
                         on_bad_lines='skip', encoding=encoding)
        print(f"Chargement réussi: {df.shape[0]} lignes et {df.shape[1]} colonnes")
        return df
    except Exception as e:
        print(f"Erreur lors du chargement: {e}")
        try:
            # Alternative avec CSV standard
            df = pd.read_csv(filepath, sep=sep, encoding=encoding)
            print(f"Chargement alternatif réussi: {df.shape[0]} lignes et {df.shape[1]} colonnes")
            return df
        except Exception as e2:
            print(f"Échec du chargement alternatif: {e2}")
            return None


df = load_data('fusionV3.csv')
df_clean = df.copy()

Chargement réussi: 7944 lignes et 46 colonnes


Delete colonnes inutiles

In [39]:
# Liste des colonnes à supprimer
colonnes_a_supprimer = [
    "titre_jpbox"
]

# Supprimer uniquement les colonnes qui existent dans le dataframe
colonnes_existantes = [col for col in colonnes_a_supprimer if col in df_clean.columns]

# Supprimer les colonnes
df_clean = df_clean.drop(columns=colonnes_existantes)

# Afficher un message indiquant quelles colonnes ont été supprimées
print(f"{len(colonnes_existantes)} colonnes ont été supprimées sur {len(colonnes_a_supprimer)} demandées.")
print(f"Colonnes supprimées: {colonnes_existantes}")
print(f"Colonnes restantes: {df_clean.columns.tolist()}")

1 colonnes ont été supprimées sur 1 demandées.
Colonnes supprimées: ['titre_jpbox']
Colonnes restantes: ['film_id', 'genre_principale', 'genres', 'date_sortie_france', 'date_sortie_usa', 'duree_minutes', 'synopsis_x', 'realisateur', 'acteurs', 'pays_origine', 'budget', 'box_office_demarrage', 'box_office_france', 'recette_usa', 'recette_monde', 'image_url', 'note_moyenne', 'titre_clean', 'titre_allocine', 'film_url', 'film_image_url', 'release_date', 'duration', 'age_classification', 'producers', 'director', 'top_stars', 'press_rating', 'viewer_rating', 'languages', 'distributor', 'year_of_production', 'film_nationality', 'filming_secrets', 'fr_entry_week', 'us_entry_week', 'fr_entries', 'us_entries', 'awards', 'associated_genres', 'press_critics_count', 'viewer_critics_count', 'broadcast_category', 'trailer_views', 'synopsis_y']


Création du jeu de données de test 

In [40]:
films_test = [
    "Zion", 
    "Dog Man", 
    "Doux Jésus", 
    "Mikado", 
    "One of Them Days", 
    "Cassandre", 
    "Le Routard",
    "Banger",
    "Minecraft, Le Film"
]

films_present = df_clean[df['titre_allocine'].isin(films_test)]

print(films_present)


     film_id    genre_principale genres date_sortie_france date_sortie_usa  \
190    24499  Comédie dramatique    NaN         09/04/2025      00/00/0000   
2965   24495               Drame    NaN         02/04/2025      00/00/0000   
4039   24399           Animation    NaN         09/04/2025      31/01/2025   
4730   24033   Aventure - Action    NaN         02/04/2025      04/04/2025   
4731   24571   Aventure - Action    NaN         09/04/2025      00/00/0000   
7089   24536             Comédie    NaN         02/04/2025      00/00/0000   
7091   24496             Comédie    NaN         02/04/2025      00/00/0000   
7092   24393             Comédie    NaN         07/04/2025      17/01/2025   
7093   24572             Comédie    NaN         09/04/2025      00/00/0000   

     duree_minutes                                         synopsis_x  \
190           34.0  Mikado et Laetitia vivent avec leurs enfants s...   
2965          43.0  Été 1998. Campagne. Cassandre a 14 ans. Dans l...   


In [41]:
# D'abord, vérifions les colonnes disponibles
print("Colonnes disponibles:", df_clean.columns.tolist())

# Trouvons une colonne qui contient les titres des films
# Examinons les premières lignes pour voir où sont stockés les titres
print(df_clean.head(3))

# En se basant sur les données partagées, les titres peuvent être dans d'autres colonnes
# Essayons de trouver des correspondances dans les colonnes synopsis_x ou synopsis_y
# qui pourraient contenir des descriptions avec les titres

# Option 1: Utiliser la colonne 'synopsis_x' ou 'synopsis_y' pour identifier les films
mask_films_test = False
for film in films_test:
    if 'synopsis_x' in df_clean.columns:
        mask_films_test = mask_films_test | df_clean['synopsis_x'].str.contains(film, case=False, na=False)
    if 'synopsis_y' in df_clean.columns:
        mask_films_test = mask_films_test | df_clean['synopsis_y'].str.contains(film, case=False, na=False)

# Option 2: Si vous avez identifié la bonne colonne pour les titres de films
# (peut-être en examinant les colonnes disponibles), utilisez celle-ci:
# Exemple si les titres sont dans une colonne appelée 'titre' ou autre chose:
titre_column = None
potential_title_columns = ['titre', 'titre_jpbox', 'film_title', 'name', 'title']
for col in potential_title_columns:
    if col in df_clean.columns:
        titre_column = col
        print(f"Colonne de titre trouvée: {col}")
        break

# Si nous avons trouvé une colonne de titre
if titre_column:
    mask_films_test = df_clean[titre_column].isin(films_test)
    print(f"Films trouvés avec {titre_column}: {df_clean[mask_films_test].shape[0]}")

# Option 3: Utiliser simplement les films d'avril 2025
print("Utilisation des films d'avril 2025 comme jeu de test...")
mask_avril_2025 = df_clean['date_sortie_france'].str.contains('04/2025', na=False)
print(f"Films d'avril 2025 trouvés: {df_clean[mask_avril_2025].shape[0]}")

# Créer le dataframe de test avec les films d'avril 2025
df_test = df_clean[mask_avril_2025].copy()
print(f"Nombre de films extraits pour le test: {len(df_test)}")

# Sauvegarder les valeurs réelles pour l'évaluation future (si disponibles)
if 'box_office_demarrage' in df_test.columns:
    box_office_reels = df_test['box_office_demarrage'].copy()
    print("Valeurs box office sauvegardées pour évaluation.")
else:
    # Créer une série vide si la colonne n'existe pas
    box_office_reels = pd.Series([None] * len(df_test), index=df_test.index)
    print("ATTENTION: Colonne box_office_demarrage non trouvée.")

# Mettre à NULL la cible dans le dataframe de test (si elle existe)
if 'box_office_demarrage' in df_test.columns:
    df_test['box_office_demarrage'] = None

# Retirer ces films du dataframe principal
df_clean = df_clean[~mask_avril_2025]
print(f"Nombre de films restants dans le dataset principal: {len(df_clean)}")

# Afficher les titres des films de test
print("\nFilms extraits pour le test:")
for i, (_, film) in enumerate(df_test.iterrows()):
    # Utiliser les informations disponibles
    film_info = []
    if 'synopsis_x' in film and isinstance(film['synopsis_x'], str) and len(film['synopsis_x']) > 10:
        film_info.append(film['synopsis_x'][:50] + "...")
    if 'genres' in film:
        film_info.append(f"Genre: {film['genres']}")
    if 'date_sortie_france' in film:
        film_info.append(f"Sortie: {film['date_sortie_france']}")
    
    box_office = box_office_reels.iloc[i] if i < len(box_office_reels) else None
    box_office_text = f"Box office: {box_office}" if box_office is not None else "Box office: Non disponible"
    
    print(f"- Film {i+1}: {' | '.join(film_info)}")
    print(f"  {box_office_text}")

Colonnes disponibles: ['film_id', 'genre_principale', 'genres', 'date_sortie_france', 'date_sortie_usa', 'duree_minutes', 'synopsis_x', 'realisateur', 'acteurs', 'pays_origine', 'budget', 'box_office_demarrage', 'box_office_france', 'recette_usa', 'recette_monde', 'image_url', 'note_moyenne', 'titre_clean', 'titre_allocine', 'film_url', 'film_image_url', 'release_date', 'duration', 'age_classification', 'producers', 'director', 'top_stars', 'press_rating', 'viewer_rating', 'languages', 'distributor', 'year_of_production', 'film_nationality', 'filming_secrets', 'fr_entry_week', 'us_entry_week', 'fr_entries', 'us_entries', 'awards', 'associated_genres', 'press_critics_count', 'viewer_critics_count', 'broadcast_category', 'trailer_views', 'synopsis_y']
                                             film_id genre_principale genres  \
0                                              20941          Fantasy    NaN   
1                                              10280          Fantasy    NaN   


Nettoyage des valeurs monétaire

In [42]:
def clean_monetary_value(value):
    """Nettoie et convertit les valeurs monétaires avec gestion améliorée des erreurs"""
    # Dictionnaire des taux de conversion approximatifs
    currency_rates = {
        '€': 1.1,  # Euro vers Dollar
        '£': 1.25, # Livre vers Dollar
        '¥': 0.0068, # Yen vers Dollar
        'CA$': 0.73, # Dollar canadien vers Dollar américain
        'A$': 0.65  # Dollar australien vers Dollar américain
    }
    
    if pd.isna(value) or value in ['Non disponible', '?', '? $', '- $', '-', '', 'nan', '? €']:
        return np.nan
        
    if isinstance(value, str):
        # Vérifier si la chaîne est un titre de film ou un texte non pertinent
        if len(value) > 20 and not any(c.isdigit() for c in value):
            return np.nan
            
        # Détecter la devise et convertir en USD
        for symbol, rate in currency_rates.items():
            if symbol in value:
                cleaned_value = value.replace(symbol, '').replace(' ', '').strip()
                try:
                    return float(cleaned_value.replace(',', '.')) * rate
                except ValueError:
                    continue
                    
        # Traitement standard pour les montants en dollars
        cleaned_value = value.replace('$', '').replace(' ', '').replace(',', '.').strip()
        
        if cleaned_value == '' or cleaned_value == '-':
            return np.nan
            
        try:
            return float(cleaned_value)
        except ValueError:
            return np.nan
            
    return value



# NETTOYAGE DES COLONNES MONÉTAIRES
monetary_columns = ['budget', 'box_office_demarrage', 'trailer_views']

for col in monetary_columns:
    if col in df_test.columns:
        print(f"Nettoyage de la colonne {col}...")
        df_test[col] = df_test[col].apply(clean_monetary_value)

if 'trailer_views' in df_test.columns:
    # Nettoyage spécifique pour trailer_views
    df_test['trailer_views_clean'] = df_test['trailer_views'].apply(
        lambda x: float(str(x).replace('vues', '').replace(',', '').strip()) 
        if isinstance(x, str) and 'vues' in x else np.nan
    )
    
    # Transformation logarithmique des vues
    df_test['log_trailer_views'] = np.log1p(df_test['trailer_views_clean'])


Nettoyage de la colonne budget...
Nettoyage de la colonne box_office_demarrage...
Nettoyage de la colonne trailer_views...


In [43]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
monetary_columns = ['budget', 'box_office_demarrage', 'trailer_views']

for col in monetary_columns:
    if col in df_test.columns:
        print(f"Nettoyage de la colonne {col}...")
        df_test[col] = df_test[col].apply(clean_monetary_value)

if 'trailer_views' in df_test.columns:
    # Nettoyage spécifique pour trailer_views
    df_test['trailer_views_clean'] = df_test['trailer_views'].apply(
        lambda x: float(str(x).replace('vues', '').replace(',', '').strip()) 
        if isinstance(x, str) and 'vues' in x else np.nan
    )
    
    # Transformation logarithmique des vues
    df_test['log_trailer_views'] = np.log1p(df_test['trailer_views_clean'])

Nettoyage de la colonne budget...
Nettoyage de la colonne box_office_demarrage...
Nettoyage de la colonne trailer_views...


Nettoyage de la durée

In [44]:
def extract_duration_minutes(duration):
    """Extrait la durée en minutes à partir de différents formats"""
    if pd.isna(duration):
        return np.nan
        
    # Si c'est déjà un nombre, le retourner
    if isinstance(duration, (int, float)):
        return float(duration)
        
    # Convertir en string pour traiter
    duration_str = str(duration)
    
    # Format '1h 32min' ou '1h32min' ou '1h32'
    match = re.search(r'(\d+)h\s*(\d*)', duration_str)
    if match:
        hours = int(match.group(1))
        minutes = 0
        if match.group(2):
            minutes = int(match.group(2))
        return hours * 60 + minutes
    
    # Format '92min' ou '92 min'
    match = re.search(r'(\d+)\s*min', duration_str)
    if match:
        return int(match.group(1))
    
    # Format simple nombre (déjà en minutes)
    try:
        return float(duration_str)
    except ValueError:
        return np.nan
    


# TRAITEMENT DE LA DUREE
if 'duration' in df_test.columns:
    df_test['duree_minutes'] = df_test['duration'].apply(extract_duration_minutes)
elif 'duree' in df_clean.columns:
    df_clean['duree_minutes'] = df_clean['duree'].apply(extract_duration_minutes)

# Création d'une colonne unique pour la durée
if 'duree_minutes' in df_clean.columns:
    df_clean['duree_film'] = df_clean['duree_minutes']


In [45]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
if 'duration' in df_test.columns:
    df_test['duree_minutes'] = df_test['duration'].apply(extract_duration_minutes)
elif 'duree' in df_test.columns:
    df_test['duree_minutes'] = df_test['duree'].apply(extract_duration_minutes)

# Création d'une colonne unique pour la durée
if 'duree_minutes' in df_test.columns:
    df_test['duree_film'] = df_test['duree_minutes']

Nettoyage des notes

In [46]:
def clean_numeric_value(value):
    """Nettoie et convertit les valeurs numériques générales"""
    if pd.isna(value) or value in ['Non disponible', '?', '-', '', 'nan']:
        return np.nan
        
    if isinstance(value, str):
        cleaned_value = value.replace(',', '.').strip()
        
        if cleaned_value == '' or cleaned_value == '-':
            return np.nan
            
        try:
            return float(cleaned_value)
        except ValueError:
            return np.nan
            
    return value



# NETTOYAGE DES NOTES
numeric_columns = ['press_rating']

for col in numeric_columns:
    if col in df_clean.columns:
        print(f"Nettoyage de la colonne {col}...")
        df_clean[col] = df_clean[col].apply(clean_numeric_value)

Nettoyage de la colonne press_rating...


In [47]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
numeric_columns = ['press_rating']

for col in numeric_columns:
    if col in df_test.columns:
        print(f"Nettoyage de la colonne {col}...")
        df_test[col] = df_test[col].apply(clean_numeric_value)

Nettoyage de la colonne press_rating...


Extraire les informations temporelles

In [48]:
def extract_month(date_str):
    """Extrait le mois à partir d'une date au format DD/MM/YYYY"""
    if pd.isna(date_str):
        return np.nan
        
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})', str(date_str))
    if match:
        return int(match.group(2))
        
    return np.nan

def extract_year(date_str):
    """Extrait l'année à partir d'une date au format DD/MM/YYYY"""
    if pd.isna(date_str):
        return np.nan
        
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})', str(date_str))
    if match:
        return int(match.group(3))
    
    # Format YYYY uniquement
    match = re.search(r'^(\d{4})$', str(date_str).strip())
    if match:
        return int(match.group(1))
        
    return np.nan

def determine_season(month):
    """Détermine la saison en fonction du mois"""
    if pd.isna(month):
        return np.nan
    month = int(month)
    if month in [12, 1, 2]:
        return 1  # Hiver
    elif month in [3, 4, 5]:
        return 2  # Printemps
    elif month in [6, 7, 8]:
        return 3  # Été
    else:
        return 4  # Automne

def is_holiday_season(month, day=15):
    """Détermine si c'est une période de vacances scolaires"""
    if pd.isna(month):
        return np.nan
    
    month = int(month)
    
    # Vacances d'été (juillet-août)
    if month in [7, 8]:
        return 1
    # Vacances de Noël (décembre)
    elif month == 12:
        return 1
    # Vacances d'hiver (février)
    elif month == 2:
        return 1
    # Vacances de printemps (avril)
    elif month == 4:
        return 1
    # Vacances de la Toussaint (octobre)
    elif month == 10:
        return 1
    else:
        return 0
    


# EXTRACTION DES INFORMATIONS TEMPORELLES
if 'date_sortie_france' in df_clean.columns:
    df_clean['annee_sortie'] = df_clean['date_sortie_france'].apply(extract_year)
    df_clean['mois_sortie'] = df_clean['date_sortie_france'].apply(extract_month)
    df_clean['saison_sortie'] = df_clean['mois_sortie'].apply(determine_season)
    df_clean['vacances_scolaires'] = df_clean['mois_sortie'].apply(is_holiday_season)

    # Créer des indicateurs pour les périodes clés de sortie
    df_clean['sortie_ete'] = (df_clean['mois_sortie'].isin([6, 7, 8])).astype(int)
    df_clean['sortie_fetes'] = (df_clean['mois_sortie'].isin([11, 12])).astype(int)
    
    # Post-COVID (après 2020)
    df_clean['post_covid'] = (df_clean['annee_sortie'] >= 2020).astype(int)
    
    # Jour de la semaine de sortie
    try:
        df_clean['day_of_week'] = pd.to_datetime(
            df_clean['date_sortie_france'], 
            format='%d/%m/%Y', 
            errors='coerce'
        ).dt.dayofweek
        
        # Est-ce une sortie mercredi (jour traditionnel en France)
        df_clean['is_wednesday_release'] = (df_clean['day_of_week'] == 2).astype(int)
        
        # Est-ce une sortie en week-end (vendredi-dimanche)
        df_clean['is_weekend_release'] = df_clean['day_of_week'].isin([4, 5, 6]).astype(int)
    except:
        print("Impossible de calculer le jour de la semaine")


In [49]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
if 'date_sortie_france' in df_test.columns:
    df_test['annee_sortie'] = df_test['date_sortie_france'].apply(extract_year)
    df_test['mois_sortie'] = df_test['date_sortie_france'].apply(extract_month)
    df_test['saison_sortie'] = df_test['mois_sortie'].apply(determine_season)
    df_test['vacances_scolaires'] = df_test['mois_sortie'].apply(is_holiday_season)

    # Créer des indicateurs pour les périodes clés de sortie
    df_test['sortie_ete'] = (df_test['mois_sortie'].isin([6, 7, 8])).astype(int)
    df_test['sortie_fetes'] = (df_test['mois_sortie'].isin([11, 12])).astype(int)
    
    # Post-COVID (après 2020)
    df_test['post_covid'] = (df_test['annee_sortie'] >= 2020).astype(int)
    
    # Jour de la semaine de sortie
    try:
        df_test['day_of_week'] = pd.to_datetime(
            df_test['date_sortie_france'], 
            format='%d/%m/%Y', 
            errors='coerce'
        ).dt.dayofweek
        
        # Est-ce une sortie mercredi (jour traditionnel en France)
        df_test['is_wednesday_release'] = (df_test['day_of_week'] == 2).astype(int)
        
        # Est-ce une sortie en week-end (vendredi-dimanche)
        df_test['is_weekend_release'] = df_test['day_of_week'].isin([4, 5, 6]).astype(int)
    except:
        print("Impossible de calculer le jour de la semaine")

Catégoriser le budget

In [50]:
def categorize_budget(budget):
    """Catégorise le budget en 4 niveaux"""
    if pd.isna(budget):
        return np.nan
    elif budget < 10000000:  # Moins de 10 millions
        return 1  # Petit budget
    elif budget < 50000000:  # Entre 10 et 50 millions
        return 2  # Budget moyen
    elif budget < 100000000:  # Entre 50 et 100 millions
        return 3  # Gros budget
    else:  # 100 millions et plus
        return 4  # Blockbuster

df_clean['budget'] = pd.to_numeric(df_clean['budget'], errors='coerce')
# Transformation logarithmique du budget
if 'budget' in df_clean.columns:
    df_clean['log_budget'] = np.log1p(df_clean['budget'])
    
    # Catégorisation du budget
    df_clean['categorie_budget'] = df_clean['budget'].apply(categorize_budget)
    
    # Blockbusters d'été
    budget_median = df_clean['budget'].median()
    df_clean['is_summer_blockbuster'] = ((df_clean['mois_sortie'] >= 6) & 
                                        (df_clean['mois_sortie'] <= 8) & 
                                        (df_clean['budget'] > budget_median)).astype(int)
    
    # Ratio marketing/budget (estimation à partir de règles de l'industrie)
    df_clean['marketing_ratio'] = np.where(
        df_clean['budget'] > df_clean['budget'].quantile(0.75), 0.5,
        np.where(df_clean['budget'] > df_clean['budget'].quantile(0.25), 0.3, 0.2)
    )
    df_clean['estimated_marketing_budget'] = df_clean['budget'] * df_clean['marketing_ratio']
    df_clean['estimated_total_budget'] = df_clean['budget'] + df_clean['estimated_marketing_budget']
    df_clean['log_total_budget'] = np.log1p(df_clean['estimated_total_budget'])
    

In [51]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
if 'budget' in df_test.columns:
    df_test['log_budget'] = np.log1p(df_test['budget'])
    
    # Catégorisation du budget
    df_test['categorie_budget'] = df_test['budget'].apply(categorize_budget)
    
    # Blockbusters d'été
    budget_median = df_test['budget'].median()
    df_test['is_summer_blockbuster'] = ((df_test['mois_sortie'] >= 6) & 
                                        (df_test['mois_sortie'] <= 8) & 
                                        (df_test['budget'] > budget_median)).astype(int)
    
    # Ratio marketing/budget (estimation à partir de règles de l'industrie)
    df_test['marketing_ratio'] = np.where(
        df_test['budget'] > df_test['budget'].quantile(0.75), 0.5,
        np.where(df_test['budget'] > df_test['budget'].quantile(0.25), 0.3, 0.2)
    )
    df_test['estimated_marketing_budget'] = df_test['budget'] * df_test['marketing_ratio']
    df_test['estimated_total_budget'] = df_test['budget'] + df_test['estimated_marketing_budget']
    df_test['log_total_budget'] = np.log1p(df_test['estimated_total_budget'])

Traitement des acteurs et franchises

In [52]:
# Caractéristique pour les films avec des stars importantes
famous_actors = [
                # Acteurs
                'Leonardo DiCaprio', 'Dwayne Johnson', 'Brad Pitt', 'Tom Cruise', 'Ryan Reynolds',
                'Robert Downey Jr', 'Chris Hemsworth', 'Chris Evans', 'Keanu Reeves', 'Will Smith',
                'Johnny Depp', 'Matt Damon', 'Christian Bale', 'Timothée Chalamet', 'Jake Gyllenhaal',
                'Benedict Cumberbatch', 'Tom Holland', 'Michael B. Jordan', 'Hugh Jackman', 'Oscar Isaac',
                'Cillian Murphy', 'Adam Driver', 'Daniel Craig', 'Joaquin Phoenix', 'Mark Wahlberg',
                'Denzel Washington', 'Tom Hanks', 'George Clooney', 'Ryan Gosling', 'Jamie Foxx',
                'Idris Elba', 'Chris Pratt', 'Samuel L. Jackson', 'Jason Statham', 'Anthony Hopkins',
                
                # Actrices
                'Margot Robbie', 'Scarlett Johansson', 'Jennifer Lawrence', 'Zendaya', 'Emma Stone',
                'Natalie Portman', 'Florence Pugh', 'Gal Gadot', 'Anya Taylor-Joy', 'Cate Blanchett',
                'Charlize Theron', 'Emily Blunt', 'Anne Hathaway', 'Jessica Chastain', 'Saoirse Ronan',
                'Viola Davis', 'Brie Larson', 'Millie Bobby Brown', 'Salma Hayek', 'Penélope Cruz',
                'Sandra Bullock', 'Angelina Jolie', 'Elizabeth Olsen', 'Rachel McAdams', 'Zoe Saldana',
                'Meryl Streep', 'Nicole Kidman', 'Julia Roberts', 'Jennifer Aniston', 'Emma Watson',
                
                # Acteurs français
                'Jean Dujardin', 'Omar Sy', 'Vincent Cassel', 'Gérard Depardieu', 'François Civil',
                'Dany Boon', 'Guillaume Canet', 'Mathieu Amalric', 'Louis Garrel', 'Melvil Poupaud',
                
                # Actrices françaises
                'Marion Cotillard', 'Léa Seydoux', 'Eva Green', 'Audrey Tautou', 'Juliette Binoche',
                'Adèle Exarchopoulos', 'Isabelle Huppert', 'Mélanie Laurent', 'Sophie Marceau', 'Catherine Deneuve'
                ]

df_clean['has_famous_actor'] = df_clean['top_stars'].apply(
                    lambda x: 1 if isinstance(x, str) and any(actor.lower() in str(x).lower() for actor in famous_actors) else 0
                )

# Feature pour les suites/franchises
if 'titre_jpbox' in df_clean.columns:
    franchise_indicators = ['2', '3', '4', '5', 'II', 'III', 'IV', 'V', 'saga', 'trilogy', 
                          'suite', 'chapitre', 'épisode', 'retour', 'episode']
    df_clean['is_franchise'] = df_clean['titre_jpbox'].apply(
        lambda x: 1 if isinstance(x, str) and any(ind in str(x).lower() for ind in franchise_indicators) else 0
    )


if 'top_stars' in df_clean.columns:
    # Nombre total d'acteurs dans le casting
    df_clean['star_count'] = df_clean['top_stars'].apply(
        lambda x: len(str(x).split(',')) if not pd.isna(x) else 0
    )
    
    # Présence d'une star majeure
    df_clean['has_famous_actor'] = df_clean['top_stars'].apply(
        lambda x: 1 if isinstance(x, str) and any(actor.lower() in str(x).lower() for actor in famous_actors) else 0
    )
    
    # Nombre d'acteurs célèbres dans le casting
    df_clean['famous_actor_count'] = df_clean['top_stars'].apply(
        lambda x: sum(1 for actor in famous_actors if isinstance(x, str) and actor.lower() in str(x).lower()) if not pd.isna(x) else 0
    )
    
    # Star power index (0-3) basé sur le nombre d'acteurs célèbres
    df_clean['star_power'] = df_clean['famous_actor_count'].apply(
        lambda x: min(3, x)  # Plafonné à 3 pour éviter les valeurs extrêmes
    )
    
    # Présence spécifique d'acteurs à forte popularité (top 10)
    top_actors = ['Leonardo DiCaprio', 'Dwayne Johnson', 'Brad Pitt', 'Tom Cruise', 
                 'Margot Robbie', 'Scarlett Johansson', 'Jennifer Lawrence', 'Robert Downey Jr',
                 'Tom Holland', 'Zendaya']
    
    df_clean['has_top_tier_actor'] = df_clean['top_stars'].apply(
        lambda x: 1 if isinstance(x, str) and any(actor.lower() in str(x).lower() for actor in top_actors) else 0
    )

In [53]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
df_test['has_famous_actor'] = df_test['top_stars'].apply(
                    lambda x: 1 if isinstance(x, str) and any(actor.lower() in str(x).lower() for actor in famous_actors) else 0
                )

# Feature pour les suites/franchises
if 'titre_jpbox' in df_test.columns:
    franchise_indicators = ['2', '3', '4', '5', 'II', 'III', 'IV', 'V', 'saga', 'trilogy', 
                          'suite', 'chapitre', 'épisode', 'retour', 'episode']
    df_test['is_franchise'] = df_test['titre_jpbox'].apply(
        lambda x: 1 if isinstance(x, str) and any(ind in str(x).lower() for ind in franchise_indicators) else 0
    )


if 'top_stars' in df_test.columns:
    # Nombre total d'acteurs dans le casting
    df_test['star_count'] = df_test['top_stars'].apply(
        lambda x: len(str(x).split(',')) if not pd.isna(x) else 0
    )
    
    # Présence d'une star majeure
    df_test['has_famous_actor'] = df_test['top_stars'].apply(
        lambda x: 1 if isinstance(x, str) and any(actor.lower() in str(x).lower() for actor in famous_actors) else 0
    )
    
    # Nombre d'acteurs célèbres dans le casting
    df_test['famous_actor_count'] = df_test['top_stars'].apply(
        lambda x: sum(1 for actor in famous_actors if isinstance(x, str) and actor.lower() in str(x).lower()) if not pd.isna(x) else 0
    )
    
    # Star power index (0-3) basé sur le nombre d'acteurs célèbres
    df_test['star_power'] = df_test['famous_actor_count'].apply(
        lambda x: min(3, x)  # Plafonné à 3 pour éviter les valeurs extrêmes
    )
    
    # Présence spécifique d'acteurs à forte popularité (top 10)
    top_actors = ['Leonardo DiCaprio', 'Dwayne Johnson', 'Brad Pitt', 'Tom Cruise', 
                 'Margot Robbie', 'Scarlett Johansson', 'Jennifer Lawrence', 'Robert Downey Jr',
                 'Tom Holland', 'Zendaya']
    
    df_test['has_top_tier_actor'] = df_test['top_stars'].apply(
        lambda x: 1 if isinstance(x, str) and any(actor.lower() in str(x).lower() for actor in top_actors) else 0
    )



Traitement des distributeurs / indicateur pour les gros studios 

Extraction des langues pour la nationalité

In [54]:
# Créer une caractéristique pour les gros distributeurs
major_distributors = ['Disney', 'Warner', 'Universal', 'Sony', 'Paramount', 'Fox', 
                         'Gaumont', 'Pathé', 'Netflix', 'Amazon', 'StudioCanal']
    
for distributor in major_distributors:
        df_clean[f'distributor_{distributor.lower()}'] = df_clean['distributor'].apply(
            lambda x: 1 if isinstance(x, str) and distributor.lower() in str(x).lower() else 0
        )
    
    # Indicateur pour les grands studios (majors)
major_studios = ['Disney', 'Warner', 'Universal', 'Sony', 'Paramount', 'Fox']
df_clean['is_major_studio'] = df_clean['distributor'].apply(
        lambda x: 1 if isinstance(x, str) and any(studio.lower() in str(x).lower() for studio in major_studios) else 0
    )

# Création et nationalités via "language"
df_clean['is_english'] = df_clean['languages'].apply(
        lambda x: 1 if isinstance(x, str) and 'anglais' in str(x).lower() else 0
    )
df_clean['is_french'] = df_clean['languages'].apply(
        lambda x: 1 if isinstance(x, str) and 'français' in str(x).lower() else 0
    )

# Création et nationalités via "film_nationality"
df_clean['is_usa'] = df_clean['film_nationality'].apply(
        lambda x: 1 if isinstance(x, str) and any(term in str(x).lower() for term in ['u.s.a', 'usa', 'états-unis']) else 0
    )
df_clean['is_france'] = df_clean['film_nationality'].apply(
        lambda x: 1 if isinstance(x, str) and 'france' in str(x).lower() else 0
    )
df_clean['is_europe'] = df_clean['film_nationality'].apply(
        lambda x: 1 if isinstance(x, str) and any(term in str(x).lower() for term in ['royaume-uni', 'allemagne', 'espagne', 'italie', 'belgique']) else 0
    )

In [55]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
major_distributors = ['Disney', 'Warner', 'Universal', 'Sony', 'Paramount', 'Fox', 
                         'Gaumont', 'Pathé', 'Netflix', 'Amazon', 'StudioCanal']
    
for distributor in major_distributors:
        df_test[f'distributor_{distributor.lower()}'] = df_test['distributor'].apply(
            lambda x: 1 if isinstance(x, str) and distributor.lower() in str(x).lower() else 0
        )
    
    # Indicateur pour les grands studios (majors)
major_studios = ['Disney', 'Warner', 'Universal', 'Sony', 'Paramount', 'Fox']
df_test['is_major_studio'] = df_test['distributor'].apply(
        lambda x: 1 if isinstance(x, str) and any(studio.lower() in str(x).lower() for studio in major_studios) else 0
    )

# Création et nationalités via "language"
df_test['is_english'] = df_test['languages'].apply(
        lambda x: 1 if isinstance(x, str) and 'anglais' in str(x).lower() else 0
    )
df_test['is_french'] = df_test['languages'].apply(
        lambda x: 1 if isinstance(x, str) and 'français' in str(x).lower() else 0
    )

# Création et nationalités via "film_nationality"
df_test['is_usa'] = df_test['film_nationality'].apply(
        lambda x: 1 if isinstance(x, str) and any(term in str(x).lower() for term in ['u.s.a', 'usa', 'états-unis']) else 0
    )
df_test['is_france'] = df_test['film_nationality'].apply(
        lambda x: 1 if isinstance(x, str) and 'france' in str(x).lower() else 0
    )
df_test['is_europe'] = df_test['film_nationality'].apply(
        lambda x: 1 if isinstance(x, str) and any(term in str(x).lower() for term in ['royaume-uni', 'allemagne', 'espagne', 'italie', 'belgique']) else 0
    )

Traitement des informations ayant une portée sur le public cible

Catégoriser par popularité

In [56]:
# Classification d'âge (restriction)
if 'age_classification' in df_clean.columns:
    df_clean['is_adult_only'] = df_clean['age_classification'].apply(
        lambda x: 1 if isinstance(x, str) and any(term in str(x).lower() 
                                              for term in ['interdit', '16 ans', '18 ans', 'adulte'])
        else 0
    )

# Nombre de critiques presse (indicateur d'attente médiatique avant sortie)
if 'press_critics_count' in df_clean.columns:
    # Extraire le nombre de critiques
    df_clean['press_critics_count_num'] = df_clean['press_critics_count'].apply(
        lambda x: int(re.search(r'(\d+)', str(x)).group(1)) if isinstance(x, str) and re.search(r'(\d+)', str(x)) else 0
    )
    
    # Catégoriser par popularité
    critics_median = df_clean['press_critics_count_num'].median()
    df_clean['press_critics_count_high'] = (df_clean['press_critics_count_num'] > critics_median).astype(int)

In [57]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
if 'age_classification' in df_test.columns:
    df_test['is_adult_only'] = df_test['age_classification'].apply(
        lambda x: 1 if isinstance(x, str) and any(term in str(x).lower() 
                                              for term in ['interdit', '16 ans', '18 ans', 'adulte'])
        else 0
    )

# Nombre de critiques presse (indicateur d'attente médiatique avant sortie)
if 'press_critics_count' in df_test.columns:
    # Extraire le nombre de critiques
    df_test['press_critics_count_num'] = df_test['press_critics_count'].apply(
        lambda x: int(re.search(r'(\d+)', str(x)).group(1)) if isinstance(x, str) and re.search(r'(\d+)', str(x)) else 0
    )
    
    # Catégoriser par popularité
    critics_median = df_test['press_critics_count_num'].median()
    df_test['press_critics_count_high'] = (df_test['press_critics_count_num'] > critics_median).astype(int)

Traitement approfondi de age_classification

In [58]:
# Classification par niveau de restriction
def get_age_rating(classification):
        if pd.isna(classification):
            return np.nan
            
        classification = str(classification).lower()
        
        if 'tous publics' in classification or 'tout public' in classification:
            return 0  # Tous publics
        elif any(term in classification for term in ['10 ans', '10+']):
            return 1  # 10+
        elif any(term in classification for term in ['12 ans', '12+']):
            return 2  # 12+
        elif any(term in classification for term in ['13 ans', 'adolescent', '13+']):
            return 2  # 13+
        elif any(term in classification for term in ['16 ans', '16+']):
            return 3  # 16+
        elif any(term in classification for term in ['18 ans', 'interdit -18', 'interdit aux mineurs', 'adulte']):
            return 4  # 18+
        else:
            return 0  # Par défaut tous publics si non spécifié
        

    # Classification binaire (tout public vs. restreint)
df_clean['is_adult_only'] = df_clean['age_classification'].apply(
        lambda x: 1 if isinstance(x, str) and any(term in str(x).lower() 
                                   for term in ['interdit', '16 ans', '18 ans', 'adulte'])
        else 0
    )
    
df_clean['age_rating'] = df_clean['age_classification'].apply(get_age_rating)
    
    # Interaction avec le genre (certains genres sont plus ou moins adaptés aux restrictions d'âge)
for genre_col in [col for col in df_clean.columns if col.startswith('genre_principale_')]:
        genre_name = genre_col.replace('genre_principale_', '')
        df_clean[f'age_rating_x_{genre_name}'] = df_clean['age_rating'] * df_clean[genre_col]
    
    # Impact sur le budget marketing (films pour adultes ont souvent un marketing différent)
if 'marketing_ratio' in df_clean.columns:
        # Pour les films adultes, un ratio marketing/budget différent
        age_factor = df_clean['age_rating'].apply(lambda x: 1 + (0.05 * x) if not pd.isna(x) else 1)
        df_clean['age_adjusted_marketing'] = df_clean['marketing_ratio'] * age_factor

One-hot encoding et interactions budget/genre

In [59]:
# One-hot encoding pour les variables catégorielles
categorical_columns = []

if 'genre_principale' in df_clean.columns:
    categorical_columns.append('genre_principale')
if 'saison_sortie' in df_clean.columns:
    categorical_columns.append('saison_sortie')
if 'categorie_budget' in df_clean.columns:
    categorical_columns.append('categorie_budget')

# One-hot encoding
if categorical_columns:
    df_clean = pd.get_dummies(df_clean, columns=categorical_columns, drop_first=False)
    
    # Lister les colonnes créées
    genre_columns = [col for col in df_clean.columns if col.startswith('genre_principale_')]
    saison_columns = [col for col in df_clean.columns if col.startswith('saison_sortie_')]
    budget_cat_columns = [col for col in df_clean.columns if col.startswith('categorie_budget_')]
    
    print(f"Colonnes de genre créées: {genre_columns}")
    print(f"Colonnes de saison créées: {saison_columns}")
    print(f"Colonnes de catégorie budget créées: {budget_cat_columns}")

# Créer des features d'interaction entre le budget et les genres
if 'budget' in df_clean.columns:
    # Utiliser seulement les genres les plus importants
    main_genres = ['action', 'comédie', 'drame', 'animation', 'fantastique', 'science-fiction']
    
    for genre in main_genres:
        genre_col = [col for col in df_clean.columns if genre.lower() in col.lower() and 'genre_' in col]
        if genre_col:
            df_clean[f'budget_x_genre_{genre}'] = df_clean['budget'] * df_clean[genre_col[0]]
    
    # Interaction entre budget et période de sortie
    if 'sortie_ete' in df_clean.columns:
        df_clean['budget_x_sortie_ete'] = df_clean['budget'] * df_clean['sortie_ete']
    if 'sortie_fetes' in df_clean.columns:
        df_clean['budget_x_sortie_fetes'] = df_clean['budget'] * df_clean['sortie_fetes']

Colonnes de genre créées: ['genre_principale_ Antonio, le si mielleux objet de l’affection naissante de Margo, et Eduardo Perez, le père d’Antonio, propriétaire du restaurant Salsa & Salsa et l’homme qui se cache peut-être derrière le masque d’El Macho, le plus impitoyable et, comme son nom l’indique, méchant macho que la terre ait jamais porté."', 'genre_principale_ Dev’Reaux, le liftier novice et Odessa, la femme de ménage belliqueuse, jouissent néanmoins d’un atout majeur : ils connaissent le bâtiment de fond en comble. Sans jamais s’en être rendu compte, ils repèrent les lieux du crime depuis des années."', 'genre_principale_ Roscoe Means, expert en explosifs ', "genre_principale_ enfin, Jeanne, prostituée, capable d'assassiner de sang froid.", 'genre_principale_- Artus | François Berléand', 'genre_principale_- Queen Latifah | Luc Besson', 'genre_principale_Adam Driver | Camille Cottin | Salma Hayek | Jeremy Irons | Jared Leto | Al Pacino | Ridley Scott', 'genre_principale_Adam San

In [60]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
# One-hot encoding pour les variables catégorielles
categorical_columns = []

if 'genre_principale' in df_test.columns:
    categorical_columns.append('genre_principale')
if 'saison_sortie' in df_test.columns:
    categorical_columns.append('saison_sortie')
if 'categorie_budget' in df_test.columns:
    categorical_columns.append('categorie_budget')

# One-hot encoding
if categorical_columns:
    df_test= pd.get_dummies(df_test, columns=categorical_columns, drop_first=False)
    
    # Lister les colonnes créées
    genre_columns = [col for col in df_test.columns if col.startswith('genre_principale_')]
    saison_columns = [col for col in df_test.columns if col.startswith('saison_sortie_')]
    budget_cat_columns = [col for col in df_test.columns if col.startswith('categorie_budget_')]
    
    print(f"Colonnes de genre créées: {genre_columns}")
    print(f"Colonnes de saison créées: {saison_columns}")
    print(f"Colonnes de catégorie budget créées: {budget_cat_columns}")

# Créer des features d'interaction entre le budget et les genres
if 'budget' in df_test.columns:
    # Utiliser seulement les genres les plus importants
    main_genres = ['action', 'comédie', 'drame', 'animation', 'fantastique', 'science-fiction']
    
    for genre in main_genres:
        genre_col = [col for col in df_test.columns if genre.lower() in col.lower() and 'genre_' in col]
        if genre_col:
            df_test[f'budget_x_genre_{genre}'] = df_test['budget'] * df_test[genre_col[0]]
    
    # Interaction entre budget et période de sortie
    if 'sortie_ete' in df_test.columns:
        df_test['budget_x_sortie_ete'] = df_test['budget'] * df_test['sortie_ete']
    if 'sortie_fetes' in df_test.columns:
        df_test['budget_x_sortie_fetes'] = df_test['budget'] * df_test['sortie_fetes']

Colonnes de genre créées: ['genre_principale_Animation', 'genre_principale_Aventure - Action', 'genre_principale_Comédie', 'genre_principale_Comédie dramatique', 'genre_principale_Drame']
Colonnes de saison créées: ['saison_sortie_2']
Colonnes de catégorie budget créées: ['categorie_budget_2.0', 'categorie_budget_4.0']


Traitement de "trailer_views" approfondi

Création d'une feature pour le nombre de prix et nominations

In [61]:
# Traitement des vues de bande-annonce (feature prédictive importante)
if 'trailer_views' in df_clean.columns:
    # Nettoyer les vues de bande-annonce en tenant compte des caractères spéciaux
    df_clean['trailer_views_clean'] = df_clean['trailer_views'].apply(
        lambda x: float(re.sub(r'[^\d.]', '', str(x))) 
        if isinstance(x, str) and any(c.isdigit() for c in str(x)) else np.nan
    )
    
    # Transformation logarithmique des vues (pour normaliser)
    df_clean['log_trailer_views'] = np.log1p(df_clean['trailer_views_clean'])
    
    # Catégorisation des vues
    trailer_views_median = df_clean['trailer_views_clean'].median()
    df_clean['high_trailer_views'] = (df_clean['trailer_views_clean'] > trailer_views_median).astype(int)

# Traitement des récompenses et nominations (comme indicateurs de qualité prévisible)
if 'awards' in df_clean.columns:
    # Extraction du nombre de prix et nominations
    def extract_awards_count(awards_text):
        if pd.isna(awards_text):
            return 0, 0
        
        awards_text = str(awards_text).lower()
        
        # Extraire le nombre de prix
        prix_match = re.search(r'(\d+)\s+prix', awards_text)
        prix_count = int(prix_match.group(1)) if prix_match else 0
        
        # Extraire le nombre de nominations
        nom_match = re.search(r'(\d+)\s+nomination', awards_text)
        nom_count = int(nom_match.group(1)) if nom_match else 0
        
        return prix_count, nom_count
    
    # Créer des colonnes pour les prix et nominations
    df_clean[['prix_count', 'nomination_count']] = pd.DataFrame(
        df_clean['awards'].apply(extract_awards_count).tolist(),
        index=df_clean.index
    )
    
    # Feature combinée (prix ont plus de poids)
    df_clean['award_score'] = df_clean['prix_count'] * 2 + df_clean['nomination_count']
    
    # Indicateur de film primé
    df_clean['is_awarded'] = (df_clean['prix_count'] > 0).astype(int)

     # Interaction avec les prix (effet multiplicateur des récompenses sur les vues)
    df_clean['trailer_views_x_awards'] = df_clean['trailer_views_clean'] * df_clean['prix_count']
        
        # Ratio prix/vues (efficacité de conversion des récompenses en vues)
    non_zero_views = df_clean['trailer_views_clean'].replace(0, np.nan)
    df_clean['awards_per_view'] = df_clean['prix_count'] / non_zero_views
        
    if 'nomination_count' in df_clean.columns:
        # Interaction avec les nominations
        df_clean['trailer_views_x_nominations'] = df_clean['trailer_views_clean'] * df_clean['nomination_count']
        
    if 'award_score' in df_clean.columns:
        # Interaction avec le score combiné awards/nominations
        df_clean['trailer_views_x_award_score'] = df_clean['trailer_views_clean'] * df_clean['award_score']
        
        # Indicateur des films à haute visibilité et reconnaissance critique
        views_threshold = df_clean['trailer_views_clean'].quantile(0.75)
        df_clean['high_visibility_acclaimed'] = ((df_clean['trailer_views_clean'] > views_threshold) & 
                                               (df_clean['award_score'] > 0)).astype(int)

In [62]:
# TRAITEMENT DU JEU DE DONNEES DE TEST
# Traitement des vues de bande-annonce
if 'trailer_views' in df_test.columns:
    # Nettoyer les vues de bande-annonce en tenant compte des caractères spéciaux
    df_test['trailer_views_clean'] = df_test['trailer_views'].apply(
        lambda x: float(re.sub(r'[^\d.]', '', str(x))) 
        if isinstance(x, str) and any(c.isdigit() for c in str(x)) else np.nan
    )
    
    # Transformation logarithmique des vues
    df_test['log_trailer_views'] = np.log1p(df_test['trailer_views_clean'])
    
    # Catégorisation des vues
    trailer_views_median = df_test['trailer_views_clean'].median()
    df_test['high_trailer_views'] = (df_test['trailer_views_clean'] > trailer_views_median).astype(int)

# Traitement des récompenses et nominations
if 'awards' in df_test.columns:
    # Extraction du nombre de prix et nominations
    def extract_awards_count(awards_text):
        if pd.isna(awards_text):
            return 0, 0
        
        awards_text = str(awards_text).lower()
        
        # Extraire le nombre de prix
        prix_match = re.search(r'(\d+)\s+prix', awards_text)
        prix_count = int(prix_match.group(1)) if prix_match else 0
        
        # Extraire le nombre de nominations
        nom_match = re.search(r'(\d+)\s+nomination', awards_text)
        nom_count = int(nom_match.group(1)) if nom_match else 0
        
        return prix_count, nom_count
    
    # Extraire les prix et nominations avec une approche plus robuste
    awards_data = df_test['awards'].apply(extract_awards_count)
    prix_data = [item[0] for item in awards_data]
    nomination_data = [item[1] for item in awards_data]
    
    # Assigner les valeurs individuellement
    df_test['prix_count'] = prix_data
    df_test['nomination_count'] = nomination_data
    
    # Feature combinée (prix ont plus de poids)
    df_test['award_score'] = df_test['prix_count'] * 2 + df_test['nomination_count']
    
    # Indicateur de film primé
    df_test['is_awarded'] = (df_test['prix_count'] > 0).astype(int)
    
    # Interaction avec les prix (effet multiplicateur des récompenses sur les vues)
    if 'trailer_views_clean' in df_test.columns:
        df_test['trailer_views_x_awards'] = df_test['trailer_views_clean'] * df_test['prix_count']
        
        # Ratio prix/vues (efficacité de conversion des récompenses en vues)
        non_zero_views = df_test['trailer_views_clean'].replace(0, np.nan)
        df_test['awards_per_view'] = df_test['prix_count'] / non_zero_views
        
        if 'nomination_count' in df_test.columns:
            # Interaction avec les nominations
            df_test['trailer_views_x_nominations'] = df_test['trailer_views_clean'] * df_test['nomination_count']
        
        if 'award_score' in df_test.columns:
            # Interaction avec le score combiné awards/nominations
            df_test['trailer_views_x_award_score'] = df_test['trailer_views_clean'] * df_test['award_score']
            
            # Indicateur des films à haute visibilité et reconnaissance critique
            views_threshold = df_test['trailer_views_clean'].quantile(0.75)
            df_test['high_visibility_acclaimed'] = ((df_test['trailer_views_clean'] > views_threshold) & 
                                                 (df_test['award_score'] > 0)).astype(int)

Préparation de la Target

In [63]:
# Transformation logarithmique de la variable cible
if 'box_office_demarrage' in df_clean.columns:
    # Vérifier le type et convertir toutes les valeurs en numériques
    df_clean['box_office_demarrage'] = pd.to_numeric(df_clean['box_office_demarrage'], errors='coerce')
    
    # Remplacer les valeurs négatives par NaN (car log ne fonctionne pas sur les nombres négatifs)
    df_clean.loc[df_clean['box_office_demarrage'] < 0, 'box_office_demarrage'] = np.nan
    
    # Appliquer la transformation logarithmique
    df_clean['log_box_office_demarrage'] = np.log1p(df_clean['box_office_demarrage'])
    
    # Afficher des informations sur les transformations
    print(f"Valeurs nulles après conversion: {df_clean['box_office_demarrage'].isna().sum()}")
    print(f"Valeurs transformées en log: {df_clean['log_box_office_demarrage'].notna().sum()}")

Valeurs nulles après conversion: 7865
Valeurs transformées en log: 66


In [64]:
# Transformation logarithmique de la variable cible
if 'box_office_demarrage' in df_test.columns:
    df_test['log_box_office_demarrage'] = np.log1p(df_test['box_office_demarrage'])

Sélection des features 

In [65]:
features_selection = {
    # 1. Features de base (durée et info film)
    'base': [col for col in df_clean.columns if col in ['duree_minutes', 'duree', 'duree_film']],
    
    # 2. Features budgétaires
    'budget': [col for col in df_clean.columns 
               if 'budget' in col.lower() 
               and 'budget_x_' not in col],
    
    # 3. Marketing
    'marketing': [col for col in df_clean.columns if 'marketing' in col.lower()],
    
    # 4. Features temporelles
    'temporel': [
        col for col in df_clean.columns 
        if any(term in col for term in [
            'mois_sortie', 'annee_sortie', 'saison_sortie', 
            'vacances_scolaires', 'sortie_ete', 'sortie_fetes', 'post_covid'
        ]) or col.startswith('saison_sortie_') or col.startswith('mois_')
    ],
    
    # 5. Features de jour de sortie
    'jour_sortie': [
        col for col in df_clean.columns 
        if 'wednesday' in col.lower() or 'weekend' in col.lower() or 'mercredi' in col.lower()
    ],
    
    # 6. Genre principal
    'genre_principal': [col for col in df_clean.columns if col.startswith('genre_principale_')],
    
    # 7. Genres associés
    'genre_associe': [
        col for col in df_clean.columns 
        if col.startswith('genre_') and not col.startswith('genre_principale_')
        and not 'budget_x_genre' in col
    ],
    
    # 8. Interaction budget-genre
    'interaction': [col for col in df_clean.columns if 'budget_x_' in col],
    
    # 9. Distributeur
    'distributeur': [
        col for col in df_clean.columns 
        if col.startswith('distributor_') or col == 'is_major_studio'
    ],
    
    # 10. Origine et langue
    'origine': [col for col in df_clean.columns 
                if col in ['is_english', 'is_french', 'is_usa', 'is_europe']],
    
    # 11. Acteurs et franchise
    'acteurs': [col for col in df_clean.columns 
                if col in ['star_count', 'has_famous_actor', 'is_franchise']],
    
    # 12. Press rating (disponible avant sortie)
    'presse': [col for col in df_clean.columns if 'press_rating' in col.lower()],
    
    # 13. Trailer views
    'trailer': [col for col in df_clean.columns 
                if 'trailer_views' in col.lower() or 'log_trailer_views' in col.lower() 
                or 'high_trailer_views' in col.lower()],
    
    # 14. Awards/nominations (provenant de festivals avant sortie générale)
    'awards': [col for col in df_clean.columns 
               if 'award' in col.lower() or 'prix_count' in col.lower() 
               or 'nomination_count' in col.lower() or 'is_awarded' in col.lower()],
    
    # 15. Public cible
    'target_audience': [col for col in df_clean.columns 
                      if col in ['is_adult_only']]
}

features_to_exclude = [
    'box_office_france', 'fr_entry_week', 'us_entry_week', 'fr_entries', 
    'note_moyenne', 'viewer_critics_count', 'viewer_rating'
]

# Aplatir la liste de features
selected_features = []
for category, features in features_selection.items():
    # Filtrer uniquement les colonnes qui existent réellement et qui ne sont pas à exclure
    existing_features = [f for f in features if f in df_clean.columns 
                         and not any(exclude in f for exclude in features_to_exclude)]
    selected_features.extend(existing_features)
    print(f"{category}: {len(existing_features)} features")

# Éliminer les doublons
selected_features = list(set(selected_features))

print(f"\nNombre total de features sélectionnées: {len(selected_features)}")

base: 2 features
budget: 6 features
marketing: 3 features
temporel: 12 features
jour_sortie: 2 features
genre_principal: 836 features
genre_associe: 0 features
interaction: 6 features
distributeur: 12 features
origine: 4 features
acteurs: 2 features
presse: 1 features
trailer: 7 features
awards: 8 features
target_audience: 1 features

Nombre total de features sélectionnées: 897


In [66]:
print(selected_features)

['has_famous_actor', 'genre_principale_Adrien Brody | Owen Wilson | Bill Murray | Natalie Portman', 'is_awarded', 'genre_principale_Christian Bale | Tom Hardy | Anne Hathaway | Michael Caine | Marion Cotillard | Morgan Freeman | Joseph Gordon-Levitt | Gary Oldman | Aaron Eckhart | Cillian Murphy', 'genre_principale_Dwayne Johnson | Sean William Scott | Christopher Walken | Arnold Schwarzenegger', 'genre_principale_Kad Merad | Benoît Poelvoorde | Zabou Breitman | Léa Drucker | Denis Podalydès', 'distributor_warner', 'genre_principale_Robert Pattinson | Kristen Stewart | Dakota Fanning | Bryce Dallas Howard | Stephenie Meyer | Howard Shore', 'genre_principale_Cameron Diaz', 'genre_principale_Melvil Poupaud | Laure Calamy', 'genre_principale_Pio Marmaï | Camille Cottin', 'genre_principale_Elodie Bouchez | Gérard Depardieu | Romain Duris', 'distributor_paramount', 'genre_principale_Chris Tucker | Jackie Chan | Yvan Attal | Julie Depardieu', 'genre_principale_Yun-Fat Chow | Brian Tyler', 'g

Préparation des données pour la modélisation

In [67]:
# Préparer X et y
X = df_clean[selected_features].copy()
y = df_clean['box_office_demarrage'].copy()
y_log = df_clean['log_box_office_demarrage'].copy()

# 1. Vérifier et corriger les colonnes non numériques
for col in X.columns:
    # Vérifier si la colonne contient des valeurs non numériques
    if X[col].dtype == 'object':
        print(f"Colonne non numérique détectée: {col}")
        
        # Tentative de conversion
        try:
            X[col] = X[col].apply(lambda x: str(x).replace(' ', '') if isinstance(x, str) else x)
            X[col] = pd.to_numeric(X[col], errors='coerce')
            print(f"  → Conversion réussie pour {col}")
        except Exception as e:
            print(f"  → Erreur lors de la conversion: {e}")
            # Supprimer les colonnes qui ne peuvent pas être converties
            X = X.drop(columns=[col])
            print(f"  → Colonne {col} supprimée")

print(f"Nombre de colonnes après nettoyage: {X.shape[1]}")

# 2. Vérifier les valeurs manquantes avant imputation
print(f"Valeurs manquantes par colonne:")
missing_values = X.isna().sum()
print(missing_values[missing_values > 0])

# 3. Imputation avec gestion d'erreurs
try:
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(X)
    print("Imputation réussie!")
except Exception as e:
    print(f"Erreur lors de l'imputation: {e}")
    # Stratégie alternative: utiliser la moyenne au lieu de la médiane
    try:
        print("Tentative avec la stratégie 'mean'...")
        imputer = SimpleImputer(strategy='mean')
        X_imputed = imputer.fit_transform(X)
        print("Imputation avec 'mean' réussie!")
    except Exception as e2:
        print(f"Erreur avec 'mean': {e2}")
        # Dernier recours: suppression des valeurs manquantes
        print("Suppression des lignes avec valeurs manquantes...")
        # Obtenez les indices des lignes sans valeurs manquantes
        complete_indices = X.dropna().index
        X = X.loc[complete_indices]
        y = y.loc[complete_indices]
        y_log = y_log.loc[complete_indices]
        X_imputed = X.values
        print(f"Données réduites à {len(X)} lignes")

# 4. Standardisation avec gestion d'erreurs
try:
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    print("Standardisation réussie!")
except Exception as e:
    print(f"Erreur lors de la standardisation: {e}")
    # Si la standardisation échoue, utilisez les données imputées sans standardisation
    X_scaled = X_imputed
    print("Utilisation des données imputées sans standardisation")

# 5. Vérifiez que X_scaled ne contient pas de NaN ou d'infini
if np.isnan(X_scaled).any() or np.isinf(X_scaled).any():
    print("Attention: X_scaled contient des NaN ou infini")
    # Remplacer les NaN et infini par 0
    X_scaled = np.nan_to_num(X_scaled)
    print("Les valeurs NaN et infini ont été remplacées par 0")

print(f"Dimensions finales de X_scaled: {X_scaled.shape}")

# 6. Train-test split
X_train, X_test, y_train_log, y_test_log = train_test_split(
    X_scaled, y_log, test_size=0.2, random_state=42
)

# Conserver également les valeurs y non logarithmiques pour l'évaluation
_, _, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

Colonne non numérique détectée: duree_film
  → Conversion réussie pour duree_film
Colonne non numérique détectée: trailer_views
  → Conversion réussie pour trailer_views
Colonne non numérique détectée: duree_minutes
  → Conversion réussie pour duree_minutes
Colonne non numérique détectée: awards
  → Conversion réussie pour awards
Nombre de colonnes après nettoyage: 897
Valeurs manquantes par colonne:
trailer_views_x_nominations    7436
trailer_views_x_award_score    7436
log_total_budget               7925
budget                         7925
duree_film                     1455
press_rating                   4005
awards_per_view                7436
estimated_marketing_budget     7925
budget_x_genre_action          7925
log_budget                     7925
trailer_views                  7931
mois_sortie                    2713
estimated_total_budget         7925
annee_sortie                   2713
budget_x_sortie_fetes          7925
vacances_scolaires             2713
budget_x_genre_coméd

Entrainement du modèle XGBoost

In [68]:
# Vérifier et nettoyer les valeurs problématiques dans y_train_log
print(f"NaN dans y_train_log: {np.isnan(y_train_log).sum()}")
print(f"Min: {y_train_log.min()}, Max: {y_train_log.max()}")

# Filtrer les entrées avec des valeurs NaN
valid_indices = ~np.isnan(y_train_log)
if valid_indices.sum() < len(y_train_log):
    print(f"Suppression de {len(y_train_log) - valid_indices.sum()} entrées avec valeurs NaN")
    X_train = X_train[valid_indices]
    y_train_log = y_train_log[valid_indices]
    y_train = y_train[valid_indices]
    
print(f"Nombre final d'entrées: {len(y_train_log)}")

# Entraînement du modèle XGBoost
print("Entraînement du modèle XGBoost...")
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42
)

xgb_model.fit(X_train, y_train_log)

NaN dans y_train_log: 6291
Min: 1.3862943611198906, Max: 7.607381425639791
Suppression de 6291 entrées avec valeurs NaN
Nombre final d'entrées: 53
Entraînement du modèle XGBoost...


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0.1, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

Gridsearch

In [69]:
#GRIDSEARCH 1 
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
import time
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Définition de la fonction d'évaluation
def evaluate_model(model, X, y_log, y_original, model_name="Modèle"):
    """Évalue un modèle et affiche les métriques"""
    # Filtrer les valeurs NaN
    valid_indices = ~np.isnan(y_log) & ~np.isnan(y_original)
    if valid_indices.sum() < len(y_log):
        print(f"Suppression de {len(y_log) - valid_indices.sum()} entrées avec NaN")
        X = X[valid_indices]
        y_log = y_log[valid_indices]
        y_original = y_original[valid_indices]
    
    # Prédictions et conversion
    y_pred_log = model.predict(X)
    y_pred = np.expm1(y_pred_log)
    
    # Calcul des métriques
    r2 = r2_score(y_original, y_pred)
    rmse = np.sqrt(mean_squared_error(y_original, y_pred))
    mae = mean_absolute_error(y_original, y_pred)
    r2_log = r2_score(y_log, y_pred_log)
    
    # Affichage des résultats
    print(f"\nRésultats du modèle {model_name}:")
    print(f"R² = {r2:.4f}, R² (log) = {r2_log:.4f}")
    print(f"RMSE = {rmse:.2f}, MAE = {mae:.2f}")
    
    return {'r2': r2, 'rmse': rmse, 'r2_log': r2_log, 'y_pred': y_pred, 'mae': mae}

# Enregistrer le temps de début
start_time = time.time()

# Pour réduire le temps de calcul, utilisez une recherche par étapes
# D'abord, optimisez max_depth et learning_rate
param_grid_step1 = {
    'max_depth': [5, 6, 7, 8],
    'learning_rate': [0.03, 0.05, 0.08]
}

# Modèle XGBoost avec les autres paramètres comme dans votre modèle original
xgb_model_grid = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42
)

# Configuration de la validation croisée
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Configuration du GridSearchCV
print("Étape 1: Optimisation de max_depth et learning_rate...")
grid_search_step1 = GridSearchCV(
    estimator=xgb_model_grid,
    param_grid=param_grid_step1,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# Exécuter la recherche étape 1
grid_search_step1.fit(X_train, y_train_log, eval_set=[(X_train, y_train_log)])

# Récupérer les meilleurs paramètres de l'étape 1
best_params_step1 = grid_search_step1.best_params_
print(f"Meilleurs paramètres de l'étape 1: {best_params_step1}")

# Étape 2: Optimiser min_child_weight et gamma avec les meilleurs max_depth et learning_rate
param_grid_step2 = {
    'min_child_weight': [2, 3, 4],
    'gamma': [0.05, 0.1, 0.15]
}

# Créer un nouveau modèle avec les meilleurs paramètres de l'étape 1
xgb_model_grid2 = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    max_depth=best_params_step1['max_depth'],
    learning_rate=best_params_step1['learning_rate'],
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42
)

print("Étape 2: Optimisation de min_child_weight et gamma...")
grid_search_step2 = GridSearchCV(
    estimator=xgb_model_grid2,
    param_grid=param_grid_step2,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# Exécuter la recherche étape 2
grid_search_step2.fit(X_train, y_train_log, eval_set=[(X_train, y_train_log)])

# Récupérer les meilleurs paramètres de l'étape 2
best_params_step2 = grid_search_step2.best_params_
print(f"Meilleurs paramètres de l'étape 2: {best_params_step2}")

# Étape 3: Optimiser subsample et colsample_bytree
param_grid_step3 = {
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Créer un nouveau modèle avec les meilleurs paramètres des étapes 1 et 2
xgb_model_grid3 = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    max_depth=best_params_step1['max_depth'],
    learning_rate=best_params_step1['learning_rate'],
    min_child_weight=best_params_step2['min_child_weight'],
    gamma=best_params_step2['gamma'],
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42
)

print("Étape 3: Optimisation de subsample et colsample_bytree...")
grid_search_step3 = GridSearchCV(
    estimator=xgb_model_grid3,
    param_grid=param_grid_step3,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# Exécuter la recherche étape 3
grid_search_step3.fit(X_train, y_train_log, eval_set=[(X_train, y_train_log)])

# Récupérer les meilleurs paramètres de l'étape 3
best_params_step3 = grid_search_step3.best_params_
print(f"Meilleurs paramètres de l'étape 3: {best_params_step3}")

# Créer le modèle final avec tous les meilleurs paramètres
final_params = {**best_params_step1, **best_params_step2, **best_params_step3}
print("\nMeilleurs paramètres combinés:")
for param, value in final_params.items():
    print(f"  {param}: {value}")

# Entraîner le modèle final
final_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    **final_params,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42
)

print("\nEntraînement du modèle final avec les meilleurs paramètres...")
final_model.fit(X_train, y_train_log, eval_set=[(X_train, y_train_log)])

# Afficher le temps total d'exécution
end_time = time.time()
print(f"Temps total de l'optimisation: {(end_time - start_time)/60:.2f} minutes")

# Évaluer d'abord le modèle original
original_model = xgb_model  # Votre modèle XGBoost original déjà entraîné
original_results = evaluate_model(original_model, X_test, y_test_log, y_test, "XGBoost Original")

# Évaluation du modèle final
final_results = evaluate_model(final_model, X_test, y_test_log, y_test, "XGBoost Final")

# Comparaison avec le modèle original
print("\n===== COMPARAISON DES MODÈLES =====")
print(f"R² Original: {original_results['r2']:.4f}, R² Optimisé: {final_results['r2']:.4f}")
improvement_r2 = ((final_results['r2'] - original_results['r2']) / abs(original_results['r2'])) * 100
print(f"Amélioration R²: {improvement_r2:.2f}%")

print(f"RMSE Original: {original_results['rmse']:.2f}, RMSE Optimisé: {final_results['rmse']:.2f}")
improvement_rmse = ((original_results['rmse'] - final_results['rmse']) / original_results['rmse']) * 100
print(f"Réduction RMSE: {improvement_rmse:.2f}%")

Étape 1: Optimisation de max_depth et learning_rate...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[0]	validation_0-rmse:1.08904
[1]	validation_0-rmse:1.07558
[2]	validation_0-rmse:1.06702
[3]	validation_0-rmse:1.05046
[4]	validation_0-rmse:1.03629
[5]	validation_0-rmse:1.02686
[6]	validation_0-rmse:1.01974
[7]	validation_0-rmse:1.01263
[8]	validation_0-rmse:1.00944
[9]	validation_0-rmse:1.00276
[10]	validation_0-rmse:0.99042
[11]	validation_0-rmse:0.98156
[12]	validation_0-rmse:0.97343
[13]	validation_0-rmse:0.97019
[14]	validation_0-rmse:0.96582
[15]	validation_0-rmse:0.96005
[0]	validation_0-rmse:1.09201
[16]	validation_0-rmse:0.95119
[1]	validation_0-rmse:1.08547
[17]	validation_0-rmse:0.94535
[2]	validation_0-rmse:1.08198
[18]	validation_0-rmse:0.93726
[3]	validation_0-rmse:1.07113
[19]	validation_0-rmse:0.93134
[4]	validation_0-rmse:1.06486
[20]	validation_0-rmse:0.92452
[5]	validation_0-rmse:1.05612
[21]	validation_0-rmse:0.91877
[6]	validation_0-rmse:1.05311
[2

In [70]:
# ⏱ Enregistrer le temps de début
start_time = time.time()

# 🔧 Grille d'hyperparamètres stratégique
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'colsample_bytree': [0.7, 0.9],
    'subsample': [0.8, 1.0]
}

# 📦 Modèle XGBoost avec early stopping intégré
xgb_model_grid = XGBRegressor(
    objective='reg:squarederror',
    min_child_weight=3,
    gamma=0.1,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    early_stopping_rounds=20
)

# 🔁 Validation croisée
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# 🔍 GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model_grid,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

# 🎯 Ensemble de validation pour early stopping
eval_set = [(X_train, y_train_log)]

print("🚀 Début de la recherche par grille...")
grid_search.fit(X_train, y_train_log, eval_set=eval_set)

# ⏱ Temps d’exécution
end_time = time.time()
print(f"🕒 Temps total de recherche: {(end_time - start_time)/60:.2f} minutes")

# ✅ Meilleurs hyperparamètres
print("🏆 Meilleurs hyperparamètres trouvés :")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"💡 Meilleur score (neg_mean_squared_error): {-grid_search.best_score_:.4f}")

# 📌 Récupérer le meilleur modèle
best_xgb_model = grid_search.best_estimator_

# 📊 Prédictions
y_pred_log = best_xgb_model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test_log)

# 🧪 Vérif NaN
print("🔍 Vérification des NaN :")
print(f"  → y_test_real contient {np.isnan(y_test_real).sum()} NaN")
print(f"  → y_pred contient {np.isnan(y_pred).sum()} NaN")

# 🧹 Nettoyage
mask = ~np.isnan(y_test_real) & ~np.isnan(y_pred)
y_test_real_clean = y_test_real[mask]
y_pred_clean = y_pred[mask]

# 📈 Calcul des métriques
rmse = np.sqrt(mean_squared_error(y_test_real_clean, y_pred_clean))
r2 = r2_score(y_test_real_clean, y_pred_clean)

# 🧾 Résultats
print("\n✅ Évaluation sur le jeu de test :")
print(f"  → RMSE : {rmse:.2f}")
print(f"  → R²   : {r2:.4f}")


🚀 Début de la recherche par grille...
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[0]	validation_0-rmse:1.10311
[1]	validation_0-rmse:1.10171
[2]	validation_0-rmse:1.09846
[3]	validation_0-rmse:1.09491
[4]	validation_0-rmse:1.09285
[5]	validation_0-rmse:1.09025
[6]	validation_0-rmse:1.08757
[7]	validation_0-rmse:1.08580
[0]	validation_0-rmse:1.09751
[8]	validation_0-rmse:1.08330
[1]	validation_0-rmse:1.09762
[9]	validation_0-rmse:1.08228
[0]	validation_0-rmse:1.09975
[10]	validation_0-rmse:1.08127
[2]	validation_0-rmse:1.09591
[11]	validation_0-rmse:1.07815
[1]	validation_0-rmse:1.09731
[3]	validation_0-rmse:1.09287
[12]	validation_0-rmse:1.07570
[2]	validation_0-rmse:1.09587
[13]	validation_0-rmse:1.07314
[4]	validation_0-rmse:1.09287
[3]	validation_0-rmse:1.09269
[0]	validation_0-rmse:1.10079
[14]	validation_0-rmse:1.07142
[5]	validation_0-rmse:1.09065
[1]	validation_0-rmse:1.09777
[4]	validation_0-rmse:1.09019
[15]	validation_0-rmse:1.07022
[6]	validation_0-rmse:1.

[36]	validation_0-rmse:1.03093
[16]	validation_0-rmse:1.05483
[27]	validation_0-rmse:1.04196
[62]	validation_0-rmse:0.95922
[37]	validation_0-rmse:1.02982
[17]	validation_0-rmse:1.05134
[28]	validation_0-rmse:1.03993
[38]	validation_0-rmse:1.02805
[18]	validation_0-rmse:1.04791
[63]	validation_0-rmse:0.95769
[29]	validation_0-rmse:1.03835
[39]	validation_0-rmse:1.02712
[32]	validation_0-rmse:1.01559
[19]	validation_0-rmse:1.04500
[40]	validation_0-rmse:1.02512
[30]	validation_0-rmse:1.03709
[64]	validation_0-rmse:0.95617
[33]	validation_0-rmse:1.01384
[20]	validation_0-rmse:1.04211
[21]	validation_0-rmse:1.04027
[22]	validation_0-rmse:1.03744
[65]	validation_0-rmse:0.95284
[23]	validation_0-rmse:1.03541
[66]	validation_0-rmse:0.95134
[31]	validation_0-rmse:1.03583
[34]	validation_0-rmse:1.01148
[67]	validation_0-rmse:0.94973
[24]	validation_0-rmse:1.03353
[68]	validation_0-rmse:0.94802
[25]	validation_0-rmse:1.03111
[35]	validation_0-rmse:1.00885[69]	validation_0-rmse:0.94656

[26]	val

In [71]:
# Vérifier les valeurs réelles
print("Valeurs de box_office_reels:")
print(box_office_reels.tolist())

# Convertir les valeurs réelles avec gestion des séparateurs de milliers
def convert_to_float(x):
    if pd.isna(x) or x == '-' or x == 'nan':
        return np.nan
    try:
        # Remplacer les espaces et les virgules par rien
        if isinstance(x, str):
            x = x.replace(' ', '').replace(',', '')
        return float(x)
    except (ValueError, TypeError):
        print(f"Impossible de convertir '{x}' en nombre")
        return np.nan

# Utiliser la fonction de conversion
y_real = pd.Series(box_office_reels).apply(convert_to_float)
print("Valeurs converties:")
print(y_real.tolist())

# Appliquer une transformation logarithmique sur les valeurs non-nulles
y_real_log = np.log1p(y_real)

# Créer un masque pour identifier les films avec des valeurs réelles valides
mask_real_values = ~pd.isna(y_real)
print(f"Nombre de films avec box office connu: {mask_real_values.sum()}")

if mask_real_values.sum() > 0:
    print(f"Utilisation de {mask_real_values.sum()} films avec box office connu pour estimer les autres")
    
    # Créer un modèle simple
    from sklearn.ensemble import RandomForestRegressor
    
    # Préparer les données pour un modèle simple
    X_simple = pd.DataFrame(index=df_test.index)
    
    # Extraire les features clés
    for feature in ['budget', 'duree_minutes', 'mois_sortie', 'annee_sortie']:
        if feature in df_test.columns:
            X_simple[feature] = pd.to_numeric(df_test[feature], errors='coerce')
    
    # Ajouter d'autres features si disponibles
    if 'trailer_views' in df_test.columns:
        X_simple['trailer_views'] = df_test['trailer_views'].apply(
            lambda x: float(str(x).replace('vues', '').replace(',', '').replace(' ', '').strip()) 
            if isinstance(x, str) and 'vues' in x else np.nan
        )
    
    # Remplir les valeurs manquantes
    X_simple = X_simple.fillna(X_simple.median())
    
    # Sélectionner les entrées avec des valeurs réelles
    X_train_simple = X_simple[mask_real_values]
    y_train_simple = y_real_log[mask_real_values]
    
    print("Features utilisées pour la prédiction:")
    print(X_simple.columns.tolist())
    print("Statistiques des features:")
    print(X_simple.describe())
    
    # Entraîner le modèle simple
    simple_model = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)
    simple_model.fit(X_train_simple, y_train_simple)
    
    # Prédire pour tous les films
    y_pred_log_simple = simple_model.predict(X_simple)
    y_pred_simple = np.expm1(y_pred_log_simple)
    

Valeurs de box_office_reels:
['25 470', None, '-', '23 762', '82 387', '-', '795 858', '-', nan, '56 406', '90 569', nan, '-']
Valeurs converties:
[25470.0, nan, nan, 23762.0, 82387.0, nan, 795858.0, nan, nan, 56406.0, 90569.0, nan, nan]
Nombre de films avec box office connu: 6
Utilisation de 6 films avec box office connu pour estimer les autres
Features utilisées pour la prédiction:
['budget', 'duree_minutes', 'mois_sortie', 'annee_sortie', 'trailer_views']
Statistiques des features:
             budget  duree_minutes  mois_sortie  annee_sortie  trailer_views
count  1.300000e+01      13.000000         13.0          13.0            0.0
mean   4.646154e+07      93.076923          4.0        2025.0            NaN
std    3.192861e+07       5.894369          0.0           0.0            NaN
min    1.400000e+07      85.000000          4.0        2025.0            NaN
25%    4.000000e+07      89.000000          4.0        2025.0            NaN
50%    4.000000e+07      92.000000          4.0 

In [72]:
for i, (_, film) in enumerate(df_test.iterrows()):
        # Trouver un identifiant pour le film
        film_title = None
        for title_col in ['titre_allocine', 'titre_clean', 'titre_jpbox', 'film_id']:
            if title_col in film and pd.notna(film[title_col]):
                film_title = film[title_col]
                break
        
        # Si aucun titre n'est trouvé, utiliser un extrait du synopsis
        if film_title is None:
            if 'synopsis_x' in film and pd.notna(film['synopsis_x']):
                film_title = film['synopsis_x'][:50] + "..."
            else:
                film_title = f"Film #{i+1}"
        
        # Valeur prédite
        box_office_predit = y_pred_simple[i]
        
        # Valeur réelle
        box_office_reel = y_real.iloc[i]
        
        # Formatage pour l'affichage
        pred_txt = f"{box_office_predit:,.0f} entrées"
        real_txt = "Non disponible" if pd.isna(box_office_reel) else f"{box_office_reel:,.0f} entrées"
        
        # Affichage
        print(f"- {film_title}")
        print(f"  - Box office prédit: {pred_txt}")
        print(f"  - Box office réel: {real_txt}")
        print("  --------------------------")
    
    # Calculer les métriques d'évaluation sur les films connus
valid_indices = mask_real_values
if valid_indices.sum() > 0:
        from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
        
        y_valid_real = y_real[valid_indices]
        y_valid_pred = y_pred_simple[valid_indices]
        
        print("\nÉvaluation des performances:")
        
        # R²
        r2 = r2_score(y_valid_real, y_valid_pred)
        print(f"R² = {r2:.4f}")
        
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_valid_real, y_valid_pred))
        print(f"RMSE = {rmse:.2f}")
        
        # MAE
        mae = mean_absolute_error(y_valid_real, y_valid_pred)
        print(f"MAE = {mae:.2f}")
        
        # Erreur relative moyenne
        rel_errors = np.abs((y_valid_pred - y_valid_real) / y_valid_real) * 100
        print(f"Erreur relative moyenne: {np.mean(rel_errors):.2f}%")
        print(f"Erreur médiane: {np.median(rel_errors):.2f}%")
else:
        print("Pas assez de valeurs réelles pour entraîner un modèle")

- Deux sœurs
  - Box office prédit: 27,009 entrées
  - Box office réel: 25,470 entrées
  --------------------------
- 24570
  - Box office prédit: 58,365 entrées
  - Box office réel: Non disponible
  --------------------------
- Mikado
  - Box office prédit: 37,762 entrées
  - Box office réel: Non disponible
  --------------------------
- Cassandre
  - Box office prédit: 26,088 entrées
  - Box office réel: 23,762 entrées
  --------------------------
- Ozi, la voix de la forêt
  - Box office prédit: 83,962 entrées
  - Box office réel: 82,387 entrées
  --------------------------
- Dog Man
  - Box office prédit: 66,260 entrées
  - Box office réel: Non disponible
  --------------------------
- Minecraft, Le Film
  - Box office prédit: 196,732 entrées
  - Box office réel: 795,858 entrées
  --------------------------
- Zion
  - Box office prédit: 27,009 entrées
  - Box office réel: Non disponible
  --------------------------
- Banger
  - Box office prédit: 66,260 entrées
  - Box office réel: